In [1]:
import requests
import pandas as pd
import pprint

In [5]:
url = "https://geo.irceline.be/sos/api/v1/{target:}/?expanded=true"

In [6]:
def gen_meta():
    # http://geo.irceline.be/sos/static/doc/api-doc/
    rep = requests.get(url.format(target='stations'))
    for rec in rep.json():
        row = rec['properties']
        ts = row['timeseries']
        for tkey in ts:
            yield {
                'siteid': row['id'],
                'serieid': tkey,
                'measureid': ts[tkey]['phenomenon']['id'],
                'categoryid': ts[tkey]['category']['id'],
                'featureid': ts[tkey]['feature']['id'],
                #'procedureid': ts[tkey]['procedure']['id'],
                'measurekey': ts[tkey]['phenomenon']['label'],
                'categorykey': ts[tkey]['category']['label'],
                'featurekey': ts[tkey]['feature']['label'],
                #'procedurekey': ts[tkey]['procedure']['label'],
                'lat': rec['geometry']['coordinates'][0],
                'lon': rec['geometry']['coordinates'][1],
            }

In [3]:
def get_meta(drop=True):
    df = pd.io.json.json_normalize(gen_meta())
    df['sitekey'] = df['featurekey'].apply(lambda x: x.split('-')[0].strip())
    df['sitename'] = df['featurekey'].apply(lambda x: "-".join(x.split('-')[1:]))
    if drop:
        df = df.drop(['categoryid', 'categorykey', 'featureid', 'featurekey'], axis=1)
    return df

In [13]:
meta = get_meta()
meta.sample(5)

,siteid,serieid,measureid,measurekey,lat,lon,sitekey,sitename
294,1159,10600,5,Particulate Matter < 10 µm,4.339710,51.348795,42R831,Berendrecht
434,1198,99905,62101,temperature,3.152402,50.945359,44M705,Roeselare
20,1046,10885,5,Particulate Matter < 10 µm,5.520313,50.931488,40GK11,Genk
327,1173,6889,38,Nitrogen monoxide,3.457624,50.616718,43N060,Havinnes
297,1710,10873,6002,Particulate Matter < 1 µm,4.380104,51.092000,42R834,Boom


In [9]:
rep = requests.get(url.format(target='timeseries'))
rep

<Response [200]>

In [11]:
pd.io.json.json_normalize(rep.json())

,id,label,extras,uom,referenceValues,station.properties.id,station.properties.label,station.geometry.coordinates,station.geometry.type,station.type,...,parameters.offering.label,parameters.feature.id,parameters.feature.label,parameters.procedure.id,parameters.procedure.label,parameters.phenomenon.id,parameters.phenomenon.label,parameters.category.id,parameters.category.label,statusIntervals
0,6522,"1,2-xylene O-xylene 6522 - btx, o-xyleen - pro...",[license],µg/m³,[],1112,41B006 - Bruxelles (Parlement UE),"[4.374388283352735, 50.838631187634185, NaN]",Point,Feature,...,"6522 - btx, o-xyleen - procedure",1112,41B006 - Bruxelles (Parlement UE),6522,"6522 - btx, o-xyleen - procedure",482,"1,2-xylene O-xylene",482,"1,2-xylene O-xylene",NaN
1,6274,"Benzene 6274 - Synspec - procedure, 40LD01 - L...",[license],µg/m³,[],1058,40LD01 - Laakdal,"[5.004864393587767, 51.10997612046704, NaN]",Point,Feature,...,6274 - Synspec - procedure,1058,40LD01 - Laakdal,6274,6274 - Synspec - procedure,20,Benzene,20,Benzene,NaN
2,6283,"Benzene 6283 - Synspec - procedure, 40LD02 - L...",[license],µg/m³,[],1059,40LD02 - Laakdal,"[5.021545977609839, 51.120384137099514, NaN]",Point,Feature,...,6283 - Synspec - procedure,1059,40LD02 - Laakdal,6283,6283 - Synspec - procedure,20,Benzene,20,Benzene,NaN
3,6487,Benzene 6487 - Airmotec 10000S plus benzeen - ...,[license],µg/m³,[],1107,40ZL01 - Zelzate,"[3.8084003239516524, 51.20314142607233, NaN]",Point,Feature,...,6487 - Airmotec 10000S plus benzeen - procedure,1107,40ZL01 - Zelzate,6487,6487 - Airmotec 10000S plus benzeen - procedure,20,Benzene,20,Benzene,NaN
4,6519,"Benzene 6519 - btx, benzeen - procedure, 41B00...",[license],µg/m³,[],1112,41B006 - Bruxelles (Parlement UE),"[4.374388283352735, 50.838631187634185, NaN]",Point,Feature,...,"6519 - btx, benzeen - procedure",1112,41B006 - Bruxelles (Parlement UE),6519,"6519 - btx, benzeen - procedure",20,Benzene,20,Benzene,NaN
5,6780,"Benzene 6780 - Airmotec 1000 - procedure, 42R8...",[license],µg/m³,[],1147,42R801 - Borgerhout,"[4.4318210469688415, 51.209662530910016, NaN]",Point,Feature,...,6780 - Airmotec 1000 - procedure,1147,42R801 - Borgerhout,6780,6780 - Airmotec 1000 - procedure,20,Benzene,20,Benzene,NaN
6,6842,"Benzene 6842 - Airmotec 1000 - procedure, 40R8...",[license],µg/m³,[],1161,40R833 - Stabroek,"[4.362260593200371, 51.32765887149557, NaN]",Point,Feature,...,6842 - Airmotec 1000 - procedure,1161,40R833 - Stabroek,6842,6842 - Airmotec 1000 - procedure,20,Benzene,20,Benzene,NaN
7,7050,"Benzene 7050 - btx, benzeen - procedure, 44N02...",[license],µg/m³,[],1202,44N029 - Houtem,"[2.582322781489737, 51.01628011113804, NaN]",Point,Feature,...,"7050 - btx, benzeen - procedure",1202,44N029 - Houtem,7050,"7050 - btx, benzeen - procedure",20,Benzene,20,Benzene,NaN
8,7080,"Benzene 7080 - Airmotec 1000 - procedure, 44R7...",[license],µg/m³,[],1207,44R701 - Gent,"[3.729298171690584, 51.058331716264874, NaN]",Point,Feature,...,7080 - Airmotec 1000 - procedure,1207,44R701 - Gent,7080,7080 - Airmotec 1000 - procedure,20,Benzene,20,Benzene,NaN
9,10602,"Black Carbon 10602 - - procedure, 44R701 - Gent","[license, statusIntervals]",µg/m³,[],1207,44R701 - Gent,"[3.729298171690584, 51.058331716264874, NaN]",Point,Feature,...,10602 - - procedure,1207,44R701 - Gent,10602,10602 - - procedure,391,Black Carbon,391,Black Carbon,"[{'lower': '20.0', 'upper': '10000.0', 'name':..."
